**LICENCE:**
This tutorial contains adaptations of material from [Programming Language Foundations in Agda](https://plfa.github.io/) by Phil Wadler and Wen Kokke. It is licensed under Creative Commons Attribution 4.0 International.

**SYNTAX:**
You can enter `→` by writing `->` and pressing TAB; pressing TAB again goes back to `->`. Similar useful combinations are:

| base form | alternate form |
|:---------:|:--------------:|
| ->  | → |
| \   | λ |
| neg | ¬ |
| top | ⊤ |
| bot | ⊥ |
| /\  | ∧ |
| \\/ | ∨ |
| iff | ⟺ |
| <   | ⟨ |
| >   | ⟩ |
| =<> | ≡⟨⟩ |
| qed | ∎ |
| N   | ℕ |
| Pi  | Π |
| Sigma | Σ |
| forall | ∀ |
| exists | ∃ |
| :: | ∷ |
| e | ε | 
| in | ∈ |

<!---

**PRO TRICKS:**
- You can ask Agda to provide the type of a closed expression. Just put the cursor nearby an expression and press SHIFT+TAB.
- You can ask Agda to compute the normal form of an expression enclosed in parantheses, such as `(λ x → x)` (already in normal form).

-->

In [116]:
module code.conn where -- from before

infixr 2 _∧_
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B
  
fst : {A B : Set} → A ∧ B → A
fst ⟨ a , _ ⟩ = a

snd : {A B : Set} → A ∧ B → B
snd ⟨ _ , b ⟩ = b

infixr 1 _∨_ 
data _∨_ (A : Set) (B : Set) : Set where
    left : A → A ∨ B
    right : B → A ∨ B

data ⊤ : Set where
  tt : ⊤
  
data ⊥ : Set where

⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()

infix 3 ¬_ -- higher priority than ∨ and ∧

¬_ : Set → Set
¬ A = A → ⊥

infixr 1 _⟺_
_⟺_ : (A : Set) (B : Set) → Set
A ⟺ B = (A → B) ∧ (B → A)

OK

In [117]:
module code.nat where
open import code.conn public

data ℕ : Set where
  zero : ℕ
  suc : ℕ → ℕ

{-# BUILTIN NATURAL ℕ #-}

infixl 5 _+_
_+_ : ℕ → ℕ → ℕ
zero + m = m
(suc n) + m = suc (n + m)

infix 4 _≡ℕ_
data _≡ℕ_ : ℕ → ℕ → Set where
  z≡z : 0 ≡ℕ 0
  s≡s : {m n : ℕ} → m ≡ℕ n → suc m ≡ℕ suc n

postulate ≡ℕ-refl : {n : ℕ} → n ≡ℕ n
postulate ≡ℕ-trans : {m n p : ℕ} → m ≡ℕ n → n ≡ℕ p → m ≡ℕ p
postulate ≡ℕ-sym : {m n : ℕ} → m ≡ℕ n → n ≡ℕ m
postulate ≡ℕ-cong : {m n : ℕ} (f : ℕ → ℕ) → m ≡ℕ n → f m ≡ℕ f n

infix  1 beginℕ_
infixr 2 _≡ℕ⟨⟩_ _≡ℕ⟨_⟩_
infix  3 _∎ℕ

beginℕ_ : {x y : ℕ} → x ≡ℕ y → x ≡ℕ y
beginℕ x≡y = x≡y

_≡ℕ⟨⟩_ : (x : ℕ) {y : ℕ} → x ≡ℕ y → x ≡ℕ y
x ≡ℕ⟨⟩ x≡y = x≡y

_≡ℕ⟨_⟩_ : (x : ℕ) {y z : ℕ} → x ≡ℕ y → y ≡ℕ z → x ≡ℕ z
x ≡ℕ⟨ x≡y ⟩ y≡z = ≡ℕ-trans x≡y y≡z

_∎ℕ : (x : ℕ) → x ≡ℕ x
x ∎ℕ = ≡ℕ-refl {x}

infix 4 _≤_
data _≤_ : ℕ → ℕ → Set where
  0≤n : {n : ℕ} → 0 ≤ n
  s≤s : {m n : ℕ} → m ≤ n → suc m ≤ suc n

OK

# Lists

````
module code.list where
open import code.nat public
````

We define the type of lists of elements of type `A`,
denoted `A *` (note the space, which is mandatory):

````
infixr 5 _∷_
data _* (A : Set) : Set where
  ε : A *
  _∷_ : A → A * → A *
````

This is an example of *parametrised type*,
because its definition depends on the parameter `A : Set`.

## Equality on lists

We can inductively define a notion of equality on lists.

````
infix 4 _≡L_
data _≡L_ {A : Set} : A * → A * → Set where
  ε≡ε : ε ≡L ε
  ∷≡∷ : {a : A} {as bs : A *} → as ≡L bs → a ∷ as ≡L a ∷ bs
````

We can easily prove some of its useful properties:

````
postulate ≡L-refl : {A : Set} {as : A *} → as ≡L as
postulate ≡L-sym : {A : Set} {as bs : A *} → as ≡L bs → bs ≡L as
postulate ≡L-trans : {A : Set} {as bs cs : A *} → as ≡L bs → bs ≡L cs → as ≡L cs
postulate ≡L-cong : {A B : Set} (f : A * → B *) {as bs : A *} → as ≡L bs → f as ≡L f bs
````

We define some notation that allows an intuitive equational reasoning.

````
infix 1 beginL_
infixr 2 _≡L⟨⟩_ _≡L⟨_⟩_
infix 3 _∎L

beginL_ : {A : Set} {x y : A *} → x ≡L y → x ≡L y
beginL x≡y = x≡y

_≡L⟨⟩_ : {A : Set} (x : A *) {y : A *} → x ≡L y → x ≡L y
x ≡L⟨⟩ x≡y = x≡y

_≡L⟨_⟩_ : {A : Set} (x : A *) {y z : A *} → x ≡L y → y ≡L z → x ≡L z
x ≡L⟨ x≡y ⟩ y≡z = ≡L-trans x≡y y≡z

_∎L : {A : Set} (x : A *) → x ≡L x
x ∎L = ≡L-refl
````

We can define standard operations on lists,
such as concatenation, map, filter, and fold.

## Concatenation

````
infixr 5 _++_
_++_ : {A : Set} → A * → A * → A *
ε ++ ys = ys
(x ∷ xs) ++ ys = x ∷ (xs ++ ys)
````

An example of concatenation:

````
_ : 0 ∷ ε ++ 1 ∷ 2 ∷ ε ≡L 0 ∷ 1 ∷ 2 ∷ ε
_ = ≡L-refl
````

The definition of `_++_` says that ε is the left unit of `_++_`.
We show an example use of induction on lists and equational reasoning
by proving that `ε` is the right unit of `_++_`.

````
ε-++ : {A : Set} → (xs : A *) → xs ++ ε ≡L xs
ε-++ ε = ≡L-refl
ε-++ (x ∷ xs) =
  beginL
    (x ∷ xs) ++ ε
  ≡L⟨⟩
    x ∷ (xs ++ ε)
  ≡L⟨ ≡L-cong (x ∷_) (ε-++ xs) ⟩
    x ∷ xs
  ∎L
````

In the base case, we have to prove `ε ++ ε ≡L ε`,
which follows from `≡L-refl` because the l.h.s. evaluates to `ε` by the definition of `_++_`.
In the inductive case, we have to prove `(x ∷ xs) ++ ε ≡L x ∷ xs`.
The first step is justified by the fact that `(x ∷ xs) ++ ε` evaluates to `x ∷ (xs ++ ε)`
by the definition of `_++_`.
We then apply the inductive hypothesis `(ε-++ xs) ∷ xs ++ ε ≡L xs`
in the context obtained by consing `x`, i.e., `(x ∷_)`.
(Here, `(x ∷_)` denotes the *section* of `_∷_` obtained by fixing its first argument to be `x`,
thus `(x ∷_) : A * → A *`.)
We achieve this with `≡L-cong`, obtaining `x ∷ (xs ++ ε) ≡L x ∷ xs`,
as required.

The second case above could also be written in just one line as

```agda
ε-++ (x ∷ xs) = ≡L-cong (x ∷_) (ε-++ xs)
```

### Associativity of concatenation

**Exercise.** Show that concatenation of lists is an associative operation:

```agda
++-assoc : {A : Set} → (xs ys zs : A *) → (xs ++ ys) ++ zs ≡L xs ++ (ys ++ zs)
++-assoc = ?
```

*Hint:* Do induction on `xs`.

## Length

````
module code.list.len where
open import code.list public

length : {A : Set} → A * → ℕ
length ε = 0
length (_ ∷ xs) = suc (length xs)
````

For example:

````
_ : length (0 ∷ 1 ∷ ε) ≡ℕ 2
_ = ≡ℕ-refl
````

### Length and concatenation

**Exercise.** Show that

```agda
++-len : {A : Set} → (as bs : A *) → length (as ++ bs) ≡ℕ length as + length bs
++-len {A} as bs = ?
```

## Map

````
module code.list.map where
open import code.list public

map : {A B : Set} → (A → B) → A * → B *
map _ ε = ε
map f (a ∷ as) = f a ∷ map f as
````

### Map and concatenation

**Exercise.** Show that `map` commutes with `_++_`, in the sense that:

```agda
map-++ : {A B : Set} (f : A → B) (xs ys : A *) → map f (xs ++ ys) ≡L map f xs ++ map f ys
map-++ = ?
```

### Map and length

**Exercise.** Show that `map` preserves the length of the list:

```agda
map-len : {A B : Set} (f : A → B) (as : A *) → length as ≡ℕ length (map f as)
```

In [124]:
module code.list.map-len where
open import code.list.map public
open import code.list.len public

map-len : {A B : Set} (f : A → B) (as : A *) → length as ≡ℕ length (map f as)
map-len _ ε = z≡z
map-len f (a ∷ as) = s≡s (map-len f as)

OK

## Filter

````
module code.list.filter where
open import code.list

data 𝔹 : Set where
  true : 𝔹
  false : 𝔹
  
-- equality on booleans
data _≡𝔹_ : 𝔹 → 𝔹 → Set where
  tt : true ≡𝔹 true
  ff : false ≡𝔹 false

filter : {A : Set} → (A → 𝔹) → A * → A *
filter _ ε = ε
filter p (a ∷ as) with p a
... | true = a ∷ filter p as
... | false = filter p as
````

This is a example of usage of the `with` construct,
which allows us to do further pattern matching on the result of evaluating `p a`.

### Filter and concatenation

**Exercise.** Show that filter and concatenation commute:

```agda
filter-++ : {A : Set} → (p : A → 𝔹) → (as bs : A *) → filter p (as ++ bs) ≡L filter p as ++ filter p bs
```

### Filter and length

**Exercise.** Show that filter cannot increase the length of the list

```agda
filter-len : {A : Set} → (p : A → 𝔹) → (as : A *) → length (filter p as) ≤ length as
```

### Filter and map

**Exercise.** Show that filter and map commute:

```agda
filter-map : {A B : Set} (p : B → 𝔹) (f : A → B) (as : A *) → filter p (map f as) ≡L map f (filter (p ∘ f) as)
```

Is the following true for some choice of the predicate `q`?
Is there any assumption on `f` that makes is true?

```agda
postulate filter-map2 : {A B : Set} (p : A → 𝔹) (f : A → B) (as : A *) → map f (filter p as) ≡L filter q (map f as)
```

````
-- here P is a total function mapping elements of A to Set
data All {A : Set} (P : A → Set) : A * → Set where
  all-ε  : All P ε
  all-∷ : {x : A} {xs : A *} → P x → All P xs → All P (x ∷ xs)


````

## Reverse

````
module code.list.rev where
open import code.list public
````

Below is the standard definition of a function that reverses the contents of a list.

````
reverse : {A : Set} → A * → A *
reverse ε = ε
reverse (a ∷ as) = reverse as ++ a ∷ ε
````

### Reverse and concatenation

**Exercise.** Show the following relationship between inverse and concatenation.

```agda
rev-++ : {A : Set} (as bs : A *) → reverse (as ++ bs) ≡L reverse bs ++ reverse as
```

*Hint*: Do induction on `as`. In the base case,
you will need the following fact proved earlier:

```agda
ε-++ : {A : Set} → (xs : A *) → xs ++ ε ≡L xs
```

In the inductive case, you will need associativity of list concatenation:

````
postulate ++-assoc : {A : Set} → (xs ys zs : A *) → (xs ++ ys) ++ zs ≡L xs ++ (ys ++ zs)
````

In [128]:
module code.list.rev-concat where
open import code.list.rev public

rev-++ : {A : Set} (as bs : A *) → reverse (as ++ bs) ≡L reverse bs ++ reverse as
rev-++ ε bs = 
  beginL
    reverse (ε ++ bs)
  ≡L⟨⟩
    reverse bs
  ≡L⟨ ≡L-sym (ε-++ (reverse bs)) ⟩
    reverse bs ++ ε
  ≡L⟨⟩
    reverse bs ++ reverse ε
  ∎L
rev-++ (a ∷ as) bs =
  beginL
    reverse ((a ∷ as) ++ bs)
  ≡L⟨⟩
    reverse (a ∷ as ++ bs)
  ≡L⟨⟩
    reverse (as ++ bs) ++ a ∷ ε
  ≡L⟨ ≡L-cong (_++ a ∷ ε) (rev-++ as bs) ⟩
    (reverse bs ++ reverse as) ++ a ∷ ε
  ≡L⟨ ++-assoc (reverse bs) (reverse as) (a ∷ ε) ⟩
    reverse bs ++ (reverse as ++ a ∷ ε)
  ≡L⟨⟩
    reverse bs ++ reverse (a ∷ as)
  ∎L

OK

### Reverse and length

**Exercise.** Show that `reverse` preserves the length of the list:

```agda
rev-len : {A : Set} (as : A *) → length (reverse as) ≡ℕ length as
rev-len = ?
```

### Reverse and map

**Exercise.** Show that `reverse` and `map` commute:

```agda
rev-map : {A B : Set} (f : A → B) (as : A *) → reverse (map f as) ≡L map f (reverse as)
rev-map = ?
```

### Reverse and filter

**Exercise.** Show that `reverse` and `filter` commute:

```agda
rev-filter : {A : Set} (p : A → 𝔹) (as : A *) → reverse (filter p as) ≡L filter p (reverse as)
rev-filter = ?
```

**Exercise.** As an application of `rev-++`,
show that the reverse operation is *involutive*, in the following sense:

```agda
rev-inv : {A : Set} → as ≡L reverse (reverse as)
```

*Hint:* In the inductive step, you need to apply `rev-++` before you can apply the inductive hypothesis.

In [129]:
module code.list.rev-inv where
open import code.list.rev public

postulate rev-++ : {A : Set} (as bs : A *) → reverse (as ++ bs) ≡L reverse bs ++ reverse as

rev-inv : {A : Set} (as : A *) → reverse (reverse as) ≡L as 
rev-inv ε = ≡L-refl
rev-inv (a ∷ as) = 
    beginL
        reverse (reverse (a ∷ as))
    ≡L⟨⟩
        reverse (reverse as ++ a ∷ ε)
    ≡L⟨ rev-++ (reverse as) (a ∷ ε) ⟩
        reverse (a ∷ ε) ++ reverse (reverse as)
    ≡L⟨⟩
        a ∷ reverse (reverse as)
    ≡L⟨ ≡L-cong (a ∷_) (rev-inv as) ⟩ 
        a ∷ as
    ∎L

OK

## Fast reverse

````
module code.list.fastrev where
open import code.list.rev public
````

The function `reverse` presented above has quadratic time complexity.
A well-known more efficient way of reversing a list uses an extra parameter,
called *accumulator*:

````
reverseAcc : {A : Set} → A * → A * → A *
reverseAcc as ε = as
reverseAcc as (x ∷ xs) = reverseAcc (x ∷ as) xs

fastReverse : {A : Set} → A * → A *
fastReverse = reverseAcc ε
````

**Exercise.** We want to prove that `reverse` and `fastReverse` are equivalent.

1. First establish a useful lemma about `reverse`:

```agda
reverse-lemma : {A : Set} → (xs ys : A *) → (x : A) → reverse (x ∷ xs) ++ ys ≡L reverse xs ++ x ∷ ys
```

*Hint:* You don't need induction, but you will need `++-assoc`.

2. Then establish the following invariant satisfied by the helper function `reverseAcc`:

```agda
invariant : {A : Set} → (xs : A *) → (as : A *) → reverse xs ++ as ≡L reverseAcc as xs
```

*Hint:* Do induction on `xs`. In the inductive case,
you will need `reverse-lemma` before you can apply the inductive hypothesis.

3. Finally, 

```agda
equivalence : {A : Set} (xs : A *) → reverse xs ≡L fastReverse xs
```

*Hint:* You do not need induction, just `ε-++` and `invariant` established above.

In [131]:
module code.list.fastrev.correct where
open import code.list.fastrev public

reverse-lemma : {A : Set} → (xs ys : A *) → (x : A) → reverse (x ∷ xs) ++ ys ≡L reverse xs ++ x ∷ ys
reverse-lemma xs ys x = beginL
    reverse (x ∷ xs) ++ ys ≡L⟨⟩
    (reverse xs ++ x ∷ ε) ++ ys ≡L⟨ ++-assoc (reverse xs) (x ∷ ε) ys ⟩
    reverse xs ++ (x ∷ ε) ++ ys ≡L⟨⟩
    reverse xs ++ x ∷ ys ∎L

invariant : {A : Set} → (xs : A *) → (as : A *) → reverse xs ++ as ≡L reverseAcc as xs
invariant ε as = ≡L-refl
invariant (x ∷ xs) as = beginL
    reverse (x ∷ xs) ++ as ≡L⟨ reverse-lemma xs as x ⟩
    reverse xs ++ (x ∷ as) ≡L⟨ invariant xs (x ∷ as) ⟩
    reverseAcc (x ∷ as) xs ≡L⟨⟩
    reverseAcc as (x ∷ xs) ∎L

equivalence : {A : Set} (xs : A *) → reverse xs ≡L fastReverse xs
equivalence xs = beginL
    reverse xs      ≡L⟨ ≡L-sym (ε-++ (reverse xs)) ⟩
    reverse xs ++ ε ≡L⟨ invariant xs ε ⟩
    reverseAcc ε xs ≡L⟨⟩
    fastReverse xs ∎L

OK

## Maximum

In [132]:
module lab00-list-length where

open import lab00-nat
open import lab00-list

length : {A : Set} → A * → ℕ
length xs = foldr _+_ 0 (map (const 1) xs)

-- first property: all the elements of the output satisfy the predicate
filter-p : {A : Set} (p : A -> Bool) (xs : A *) -> all p (filter p xs) ≡ true
filter-p p ε = refl
filter-p p (x :: xs) with p x
... | true = ?
... | false = ?

filter-p' : {A : Set} (p : A -> Bool) (xs : A *) -> All (\(a : A) -> T (p a)) (filter p xs)
filter-p' p ε = all-ε
filter-p' p (a :: as) with p a
... | true = all-:: ⊤ (filter-p' p as)
-- ⊥
... | false = ?

*Type-checking* 
*Type-checking* Checking lab00-list-length (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-list-length.agda).

*Error* /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-list-length.agda:3,1-22
Failed to find source of module lab00-nat in any of the following
locations:
  /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-nat.agda
  /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/lab00-nat.lagda
  /usr/local/lib/agda-stdlib/src/lab00-nat.agda
  /usr/local/lib/agda-stdlib/src/lab00-nat.lagda
  /Users/lorenzo/.cabal/share/x86_64-osx-ghc-8.6.4/Agda-2.6.0/lib/prim/lab00-nat.agda
  /Users/lorenzo/.cabal/share/x86_64-osx-ghc-8.6.4/Agda-2.6.0/lib/prim/lab00-nat.lagda
when scope checking the d

# Intuitionistic first-order  logic

We show how dependent types can be used to implement universal quantification (dependent function space)
and existential quantification (dependent product),
thus concluding our overview of intuitionistic logic.

## The universal quantifier

````
module code.universal where
````

In intuitionistic logic a proof of $\forall (a : A) B$ is a function $f$ mapping a proof $a$ of $A$
into a proof $f\; a$ of $B\; a$, where we can see $B$ as a family of types indexed by proofs of $A$.
The universal quantifier is implemented via the *dependent function space*
````
Π : (A : Set) → (B : A → Set) → Set
Π A B = (a : A) → B a
````
This generalises implication `A → B`, which corresponds to non-dependent function space.
In this sense, in intuitionistic logic implication is a special case of universal quantification.

Universal quantification also generalises conjunction,
since the type $B_1 \wedge B_2$ is isomorphic to $\Pi\; A\; B$
where $A = \{1, 2\}$ and $B = \{ 1 \mapsto B_1, 2 \mapsto B_2 \}$.
For this reason, sometimes `Π` is called dependent product (hence the notation).

In [134]:
module code.universal where

Π : (A : Set) → (B : A → Set) → Set
Π A B = (a : A) → B a

-- compare with implication:
-- ((a : A) → B a) versus A → B

-- the type of the first argument of Π can be inferred from the second
forAll : {A : Set} → (B : A → Set) → Set
forAll {A} B = Π A B

-- we introduce a convenient syntax reminiscent of universal quantification in logic
∀-syntax = forAll
infix 0 ∀-syntax
syntax ∀-syntax (λ x → B) = ∀[ x ] B

-- dependent apply; corresponds to ∀-elimination
apply : {A : Set} → {B : A → Set} → Π A B → (a : A) → B a
apply f x = f x

OK

We show an application of universal quantification
and prove that every natural number is either even or odd:
```agda
even∨odd : ∀[ n ] Even n ∨ Odd n
```

We now explain how the syntax above works.
The latter expands to 
```agda
even∨odd : forAll (λ n → Even n ∨ Odd n)
```
which in turn is the same as `Π A B`
with `A = ℕ` and `B = λ n → Even n ∨ Odd n`
(we can avoid typing `n` because its occurrence in `Even n` implies `n : ℕ`), i.e.,
```agda
even∨odd : Π ℕ (λ n → Even n ∨ Odd n)
```
By unrolling the definition of `Π A B = (a : A) → B a`,
we finally get
```agda
even∨odd : (n : ℕ) → Even n ∨ Odd n
```
This shows precisely how the universal quantifier is implemented by having `B a` to depend on `(a : A)`.
Thus, while `∀[ n ] Even n ∨ Odd n` is just syntactic sugaring for the more familiar `(n : ℕ) → Even n ∨ Odd n`,
we will use the former when we want to emphasise its logical content,
and the latter when we want to emphasise its computational content.
As a courtesy of the Curry-Howard isomorphism, they correspond to each other.

We now proceed to prove `even∨odd`,
which is another instance of external verification.
The proof is by induction on `n`.
In the base case `n = 0` and we can use the base constructor `zero : Even zero`
(notice how `zero` is overloaded):
```agda
even∨odd zero = left zero
```
In the inductive case, we proceed as follows:
```agda
even∨odd (suc n) with even∨odd n
... | left even = right (suc even)
... | right odd = left (suc odd)
```

### The `with` pattern matching construct

In order to decide whether `suc n` is even or odd,
we recursively call `even∨odd n` and inspect its result:
if it pattern matches with `left even` (where `even : Even n`)
then we return `right (suc even)` (where `suc even : Odd (suc n)`, again `suc` is overloaded),
and symmetrically in the other case.
This is the first time that we see the `with` construct,
which allows us to extend the set of arguments that we can pattern match on.

In [135]:
module code.nat.even-odd-total where
open import code.universal
open import code.nat.even-odd
open import code.conn

-- with syntax sugaring
even∨odd : ∀[ n ] Even n ∨ Odd n -- the same as: Π ℕ (λ n → Even n ∨ Odd n)
even∨odd zero = left zero
even∨odd (suc n) with even∨odd n
... | left even = right (suc even)
... | right odd = left (suc odd)

OK

**Exercise.** Show that the order `_≤_` is total, in the sense that
```agda
≤-total : ∀[ m ] ∀[ n ] m ≤ n ∨ n ≤ m
```
*Hint:* Induction on `m`.

In [136]:
module code.universal.example where
open import code.nat.leq
open import code.universal
open import code.conn public

≤-total : ∀[ m ] ∀[ n ] m ≤ n ∨ n ≤ m
≤-total = ?

*Type-checking* 
*Type-checking* Checking code.universal.example (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/universal/example.agda).

*Type-checking*   Checking code.eq (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda).

*Error* /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda:41,15-16
Not in scope:
  *
  at /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda:41,15-16
when scope checking *

### The universal quantifier, ∧, and ∨

**Exercise.**
Which of the following are intuitionistic tautologies?
Which are classic tautologies?
Which are none?
Prove the intuitionistic ones.

1. $\forall a B \wedge C \to (\forall a B) \wedge (\forall a C)$.
2. $(\forall a B) \wedge (\forall a C) \to \forall a B \wedge C$.
3. $\forall a B \vee C \to (\forall a B) \vee (\forall a C)$.
4. $ \forall a B \vee C \to (\forall a B) \vee C$, where $a$ does not occurr in $C$.
5. $(\forall a B) \vee (\forall a C) \to \forall a (B \vee C)$.

*Hint:* If you cannot easily program a solution, then most likely there is no solution.

In [138]:
module code.universal.ex1 where
open import code.universal
open import code.conn

-- 1.
∀∧-distr : {A : Set} {B C : A → Set} → ∀[ a ] B a ∧ C a → (∀[ a ] B a) ∧ (∀[ a ] C a)
∀∧-distr = ?

-- 2.
∧∀-distr : {A : Set} {B C : A → Set} → (∀[ a ] B a) ∧ (∀[ a ] C a) → ∀[ a ] B a ∧ C a
∧∀-distr = ?

-- 3.
∀∨-distr : {A : Set} {B C : A → Set} → ∀[ a ] B a ∨ C a → (∀[ a ] B a) ∨ (∀[ a ] C a)
∀∨-distr = ?

-- 4.
∀∨-distr' : {A C : Set} {B : A → Set} → ∀[ a ] B a ∨ C → (∀[ a ] B a) ∨ C
∀∨-distr' = ?

-- 5.
∨∀-distr : {A : Set} {B C : A → Set} → (∀[ a ] B a) ∨ (∀[ a ] C a) → ∀[ a ] B a ∨ C a
∨∀-distr = ?

?0 : ∀-syntax (λ a → B a ∧ C a) → ∀-syntax B ∧ ∀-syntax C
?1 : ∀-syntax B ∧ ∀-syntax C → ∀-syntax (λ a → B a ∧ C a)
?2 : ∀-syntax (λ a → B a ∨ C a) → ∀-syntax B ∨ ∀-syntax C
?3 : ∀-syntax (λ a → B a ∨ C) → ∀-syntax B ∨ C
?4 : ∀-syntax B ∨ ∀-syntax C → ∀-syntax (λ a → B a ∨ C a)


## The existential quantifier

In intuitionistic logic,
a proof of $\exists (a : A) B$ is a pair $(a, b)$,
where $a$ is a proof of $A$
and $b$ is a proof of $B\; a$.
Like in universal quantification, we can see $B$ as a family of types indexed by proofs of $A$.
The existential quantifier is implemented with the *dependent product*:
```agda
data Σ (A : Set) (B : A → Set) : Set where
    ⟨_,_⟩ : (a : A) → B a → Σ A B
```
Compare this with conjunction, which corresponds to non-dependent product $A \wedge B$:
```agda
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B
```
In this sense, in intuitionistic logic existential quantification `Σ` generalises conjunction,
which justifies the name dependent product.
(This can create confusion because `Π` is sometimes called dependent product too,
since also `Π` generalises conjunction.)

Existential quantification also generalises disjunction,
since the type $B_1 \vee B_2$ is isomorphic to $\Sigma\; A\; B$
with $A = \{1, 2\}$ and $B = \{ 1 \mapsto B_1, 2 \mapsto B_2 \}$.
For this reason, `Σ` is sometimes called dependent sum.

In [140]:
module code.existential where

-- the only exists-introduction rule
data Σ (A : Set) (B : A → Set) : Set where
    ⟨_,_⟩ : (a : A) → B a → Σ A B

-- compare with conjunction:
-- ⟨_,_⟩ : (a : A) → B a → Σ A B versus
-- ⟨_,_⟩ : A → B → A ∧ B

thereExists : ∀ {A : Set} (B : A → Set) → Set
thereExists {A} B = Σ A B

∃-syntax = thereExists
infix 0 ∃-syntax
syntax ∃-syntax (λ x → B) = ∃[ x ] B

-- aka uncurry
∃-elim : {A : Set} {B : A → Set} {C : Set} → ((a : A) → B a → C) → Σ A B → C
∃-elim a→b→c ⟨ a , b ⟩ = a→b→c a b

OK

### An example

As an application of existential quantification,
we show that `m ≤ n` implies that there is some `p` s.t. `m + p ≡ n`:
```agda
q : ∀[ m ] ∀[ n ] (m ≤ n → ∃[ p ] m + p ≡ n)
```

The proof is by induction on `m` (which is also simultaneously an induction on `m ≤ n`).
In the base case, `m ≡ 0` and we can just take as witness for `p` the value `n`:

```agda
q zero n _ = ⟨ n , refl ⟩
```

In the inductive case, we have

```agda
q (suc m) (suc n) (s≤s m≤n) with q m n m≤n
... | ⟨ p , m+p≡n ⟩ = ⟨ p , cong suc m+p≡n ⟩
```

We recursively call `q m n m≤n` to collect a witness `⟨ p , m+p≡n ⟩ : ∃[ p ] m + p ≡ n`,
where `p : ℕ` and `m+p≡n : m + p ≡ n`.
Then, `⟨ p , cong suc m+p≡n ⟩ : ∃[ p ] suc m + p ≡ suc n`
because `suc m + p` normalises to `suc (m + p)`.

We do not have to worry about the other case `q (suc m) zero (s≤s m≤n)`
because Agda correctly detects that it cannot possibly occurr as witnessed by `s≤s m≤n`.

This is another yet example of external verification.

In [141]:
module code.nat.leq-exists where
open import code.universal
open import code.existential
open import code.eq
open import code.nat.leq

q : ∀[ m ] ∀[ n ] (m ≤ n → ∃[ p ] m + p ≡ n)
q zero n _ = ⟨ n , refl ⟩
q (suc m) (suc n) (s≤s m≤n) with q m n m≤n
... | ⟨ p , m+p≡n ⟩ = ⟨ p , cong suc m+p≡n ⟩

*Type-checking* 
*Type-checking* Checking code.nat.leq-exists (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/nat/leq-exists.agda).

*Type-checking*  Checking code.existential (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/existential.agda).

*Type-checking*  Checking code.eq (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda).

*Error* /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda:41,15-16
Not in scope:
  *
  at /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda:41,15-16
when scope checking *

**Exercise.** Show the other direction, namely,
```agda
r : ∀[ m ] ∀[ n ] (∃[ p ] m + p ≡ n → m ≤ n)
```
Proceed by induction on `m`.
In the inductive case, use the fact that `suc` is injective (`suc-injective`) to derive `m + p ≡ n`,
which can be used to call `r` recursively.

In [142]:
module code.nat.leq-exists2 where
open import code.universal
open import code.existential
open import code.eq
open import code.nat.leq

suc-injective : ∀[ m ] ∀[ n ] (suc m ≡ suc n → m ≡ n)
suc-injective m n refl = refl

r : ∀[ m ] ∀[ n ] (∃[ p ] m + p ≡ n → m ≤ n)
r zero _ _ = ?
r (suc m) (suc n) ⟨ p , sucm+p≡sucn ⟩ = ?
    where
        m+p≡n : m + p ≡ n
        m+p≡n = ?
        m≤n : m ≤ n
        m≤n = ?

*Type-checking* 
*Type-checking* Checking code.nat.leq-exists2 (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/nat/leq-exists2.agda).

*Type-checking*  Checking code.eq (/Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda).

*Error* /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda:41,15-16
Not in scope:
  *
  at /Users/lorenzo/Dropbox/Workspace/teaching/Teaching/2018-2019/summer semester/LDI (logika dla informatyków)/lab/agda/code/eq.agda:41,15-16
when scope checking *

### The existential quantifier, ∧, and ∨

**Exercise.**
Establish whether the following are intuitionistic tautologies
and prove it in Agda for the positive cases:

1. $\exists a B \vee C \to (\exists a B) \vee (\exists a C)$.
2. $(\exists a B) \vee (\exists a C) \to \exists a B \vee C$.
3. $\exists a B \wedge C \to (\exists a B) \wedge (\exists a C)$.
4. $(\exists a B) \wedge (\exists a C) \to \exists a B \wedge C$.
5. $(\exists a B) \wedge C \to \exists a B \wedge C$, where $a$ does not occurr in $C$.

In [144]:
module code.existential.prop where
open import code.existential
open import code.conn

-- 1
∃∨-distr : {A : Set} {B C : A → Set} → ∃[ a ] B a ∨ C a → (∃[ a ] B a) ∨ (∃[ a ] C a)
∃∨-distr = ?

-- 2
∨∃-distr : {A : Set} {B C : A → Set} → (∃[ a ] B a) ∨ (∃[ a ] C a) → ∃[ a ] B a ∨ C a
∨∃-distr = ?

-- 3
∃∧-distr : {A : Set} {B C : A → Set} → ∃[ a ] B a ∧ C a → (∃[ a ] B a) ∧ (∃[ a ] C a)
∃∧-distr = ?

-- 4
∧∃-distr : {A : Set} {B C : A → Set} → (∃[ a ] B a) ∧ (∃[ a ] C a) → ∃[ a ] B a ∧ C a
∧∃-distr = ?

-- 5
∧∃-distr' : {A : Set} {B : A → Set} {C : Set} → (∃[ a ] B a) ∧ C → ∃[ a ] B a ∧ C
∧∃-distr' = ?

?0 : ∃-syntax (λ a → B a ∨ C a) → ∃-syntax B ∨ ∃-syntax C
?1 : ∃-syntax B ∨ ∃-syntax C → ∃-syntax (λ a → B a ∨ C a)
?2 : ∃-syntax (λ a → B a ∧ C a) → ∃-syntax B ∧ ∃-syntax C
?3 : ∃-syntax B ∧ ∃-syntax C → ∃-syntax (λ a → B a ∧ C a)
?4 : ∃-syntax B ∧ C → ∃-syntax (λ a → B a ∧ C)


### Universal quantification, existential quantification, and negation

**Exercise.** Which of the following hold in intuitionistic logic? Prove it.

1. $\exists a \forall b C \to \forall b \exists a C$, where $C$ depends on $a$ and $b$.
2. $\exists a \neg B \to \neg \forall a B$, where $B$ depends on $a$.
3. $\neg \forall a B \to \exists a \neg B$, where $B$ depends on $a$.


In [146]:
module code.universal-existential where
open import code.universal
open import code.existential
open import code.conn

-- 1
-- B cannot depend on (a : A) for the swap to be possible!
∃∀-distr : {A : Set} {B : Set} {C : A → B → Set} → ∃[ a ] ∀[ b ] C a b → ∀[ b ] ∃[ a ] C a b
∃∀-distr = ?

-- 2
¬∃→∀¬ : {A : Set} {B : A → Set} → ∃[ a ] ¬ B a → ¬ (∀[ a ] B a)
¬∃→∀¬ = ?

-- 3
¬∀→∃¬ : {A : Set} {B : A → Set} → ¬ (∀[ a ] B a) → ∃[ a ] ¬ B a
¬∀→∃¬ = ?

?0
  : ∃-syntax (λ a → ∀-syntax (C a)) →
    ∀-syntax (λ b → ∃-syntax (λ a → C a b))
?1 : ∃-syntax (λ a → ¬ B a) → ¬ ∀-syntax B
?2 : ¬ ∀-syntax B → ∃-syntax (λ a → ¬ B a)


# More on lists

## Membership

````
module code.list.member where
open import code.list public
open import code.list.len public
open import code.list.map public
open import code.list.filter public
open import code.list.rev public
open import code.existential public
````

We define an inductive predicate on lists representing the fact that an element appears in a list.

````
infix 10 _∈_
data _∈_ {A : Set} : A → A * → Set where
  stop : {a : A} {as : A *} → a ∈ (a ∷ as)
  skip : {a b : A} {as : A *} → a ∈ as → a ∈ (b ∷ as)
````

<!-- need equality to do this one
### An alternative definition

We define the following *recursive* function mapping lists to types (!).
This is called *type-level programming*.

```agda
_inside_ : {A : Set} → A → A * → Set
_ inside ε = ⊥
a inside (b ∷ as) = a ≡ b ∨ a inside as
```agda
-->

### Membership and concatenation

````
postulate ∈-++ : {A : Set} {a : A} {as bs : A *} → (a ∈ (as ++ bs)) ⟺ (a ∈ as ∨ a ∈ bs)
````

### Membership and length

````
postulate ∈-len : {A : Set} {as : A *} → 1 ≤ length as ⟺ (∃[ a ] a ∈ as)
````

### Membership and map

````
postulate ∈-map : {A B : Set} {as : A *} {a : A} (f : A → B) → a ∈ as → f a ∈ map f as
````

When is the converse direction true?

### Membership and filter

````
postulate ∈-filter : {A : Set} {as : A *} {a : A} (p : A → 𝔹) → p a ≡𝔹 true ⟺ a ∈ filter p as
````

### Membership and reverse

````
postulate ∈-rev : {A : Set} {as : A *} {a : A} → a ∈ as ⟺ a ∈ reverse as
````

## All

````
module code.list.all where
open import code.list.member public
open import code.conn public
open import code.universal public
````

````
data All {A : Set} (P : A → Set) : A * → Set where
  all-ε : All P ε
  all-∷ : {a : A} {as : A *} → P a → All P as → All P (a ∷ as)
````

### An alternative definition

We define the following recursive function mapping lists to types.
This technique is called *type-level programming*.

````
All-∧ : {A : Set} → (A → Set) → A * → Set
All-∧ _ ε = ⊤
All-∧ P (a ∷ as) = P a ∧ All-∧ P as

const : {A B : Set} → B → A → B
const b _ = b

All≡All-∧ : {A : Set} → (P : A → Set) → (as : A *) → All P as ⟺ All-∧ P as
All≡All-∧ P ε = ⟨ const tt , const all-ε ⟩
All≡All-∧ P (a ∷ as) = ⟨ f , g ⟩ where
  f : All P (a ∷ as) → All-∧ P (a ∷ as)
  f (all-∷ Pa Pas) = ⟨ Pa , fst (All≡All-∧ P as) Pas ⟩
  g : All-∧ P (a ∷ as) → All P (a ∷ as)
  g ⟨ Pa , Pas ⟩ = all-∷ Pa (snd (All≡All-∧ P as) Pas)
````

### All and concatenation

````
postulate All-++ : {A : Set} (P : A → Set) (as bs : A *) → All P (as ++ bs) ⟺ All P as ∧ All P bs
````

### All and map

### All and filter

### All and reverse

### All and membership

````
postulate All-∈ : {A : Set} {P : A → Set} (as : A *) → All P as ⟺ (∀[ a ] (a ∈ as → P a))
````

## Some

````
module code.list.some where
open import code.list.member public
open import code.conn public
open import code.existential public
````

````
data Some {A : Set} (P : A → Set) : A * → Set where
  found : {a : A} {as : A *} → P a → Some P (a ∷ as) 
  skip : {a : A} {as : A *} → Some P as → Some P (a ∷ as)
  
Some-∨ : {A : Set} → (A → Set) → A * → Set
Some-∨ _ ε = ⊥
Some-∨ P (a ∷ as) = P a ∨ Some-∨ P as

const : {A B : Set} → B → A → B
const b _ = b

Some≡Some-∨ : {A : Set} → (P : A → Set) → (as : A *) → Some P as ⟺ Some-∨ P as
Some≡Some-∨ P ε = ⟨ f , (λ x → ⊥-elim x) ⟩ where
  f : Some P ε → Some-∨ P ε
  f ()
Some≡Some-∨ P (a ∷ as) = ⟨ f , g ⟩ where
  f : Some P (a ∷ as) → Some-∨ P (a ∷ as)
  f (found Pa) = left Pa
  f (skip Pas) = right (fst (Some≡Some-∨ P as) Pas)
  g : Some-∨ P (a ∷ as) → Some P (a ∷ as)
  g (left Pa) = found Pa
  g (right Pas) = skip (snd (Some≡Some-∨ P as) Pas)
````

### Some and concatenation

````
postulate Some-++ : {A : Set} (P : A → Set) (as bs : A *) → Some P (as ++ bs) ⟺ Some P as ∨ Some P bs
````

### Some and map

### Some and filter

### Some and reverse

### Some and membership

````
postulate Some-∈ : {A : Set} {P : A → Set} (as : A *) → Some P as ⟺ (∃[ a ] a ∈ as ∧ P a)
````